In [1]:

import os
import pandas as pd
import requests
import json

In [ ]:
urls = {
    "commodities": "https://apps.fas.usda.gov/OpenData/api/esr/commodities",
    "units": "https://apps.fas.usda.gov/OpenData/api/esr/unitsOfMeasure",
    "regions": "https://apps.fas.usda.gov/OpenData/api/esr/regions",
    "countries": "https://apps.fas.usda.gov/OpenData/api/esr/countries"
       }

In [18]:
key = os.getenv("usda_api_key")
headers = {"API_KEY": key,
          "Accept": "application/json"}

In [ ]:
commodities = requests.get(commodities_url, headers=headers)
commodities = json.loads(commodities.text)

In [ ]:
json_dict = dict()

for k, v in urls.items():
    json_dict[k] = json.loads(requests.get(v, headers=headers).text)

In [ ]:
json_dict.keys()

In [4]:
data_release_url = "https://apps.fas.usda.gov/OpenData/api/esr/datareleasedates"
dr = requests.get(data_release_url, headers=headers)

In [ ]:
market_url = "https://apps.fas.usda.gov/OpenData/api/esr/exports/commodityCode/801/allCountries/marketYear/2023"

market = requests.get(market_url, headers=headers)

In [ ]:
market_json = json.loads(market.content)

In [ ]:
for k, v in json_dict.items():
    with open(f"{k}.json", "w") as w:
        w.write(json.dumps(v, indent=4))   

In [9]:
data_release = json.loads(dr.text)

with open("data_release_dates.json", "w") as w:
        w.write(json.dumps(data_release, indent=4))   

In [14]:
drd = pd.read_json("data_release_dates.json")

In [15]:
#drd["releaseTimeStamp"] = drd["releaseTimeStamp"].str.replace("T", " ", regex=False)

date_cols=["marketYearStart", "marketYearEnd", "releaseTimeStamp"]
for col in date_cols:
    drd[col] = pd.to_datetime(drd[col], format="%Y-%m-%dT%H:%M:%S")

In [17]:
drd[["commodityCode", "marketYear"]].drop_duplicates()

,commodityCode,marketYear
0,101,2023
1,102,2023
2,103,2023
3,104,2023
4,105,2023
...,...,...
1028,1602,1998
1029,1603,1998
1030,1606,1998
1031,1607,1998


In [26]:
from pathlib import Path, PurePosixPath

In [27]:
data_path = Path(__file__).parent.resolve() / "data"

NameError: name '__file__' is not defined

In [25]:
from pathlib import Path, PurePosixPath
data_path = Path(__file__).parent.resolve() / "data"

cc = 101
year = 2022
url = f"https://apps.fas.usda.gov/OpenData/api/esr/exports/commodityCode/{cc}/allCountries/marketYear/{year}"
print(url)
r = requests.get(url, headers=headers)
r_json = json.loads(r.content)
    
with open(f"{cc}_{year}.json", "w") as w:
    w.write(json.dumps(r_json, indent=4)) 

https://apps.fas.usda.gov/OpenData/api/esr/exports/commodityCode/101/allCountries/marketYear/2022


In [29]:
# Specify country
# https://apps.fas.usda.gov/OpenData/api/esr/exports/commodityCode/101/countryCode/1/marketYear/2022

# All countries
# 'https://apps.fas.usda.gov/OpenData/api/esr/exports/commodityCode/101/allCountries/marketYear/2022'

# Try with commodity 101
data_path = Path("C:/Users/daniel/Documents/ag_export_project/de_ag_export/data/")
cc = 101
drd_101 = drd.loc[drd["commodityCode"] == cc]
for year in drd_101["marketYear"].values:
    url = f"https://apps.fas.usda.gov/OpenData/api/esr/exports/commodityCode/{cc}/allCountries/marketYear/{year}"
    print(url)
    r = requests.get(url, headers=headers)
    r_json = json.loads(r.content)
    
    with open(data_path / f"{cc}_{year}.json", "w") as w:
        w.write(json.dumps(r_json, indent=4))   
    break

https://apps.fas.usda.gov/OpenData/api/esr/exports/commodityCode/101/allCountries/marketYear/2023


In [35]:
r_json[0]["dataReleaseDate"] 

{'commodityCode': 101,
 'countryCode': 2010,
 'weeklyExports': 31539,
 'accumulatedExports': 31539,
 'outstandingSales': 139595,
 'grossNewSales': 92379,
 'currentMYNetSales': 2039,
 'currentMYTotalCommitment': 171134,
 'nextMYOutstandingSales': 0,
 'nextMYNetSales': 0,
 'unitId': 1,
 'weekEndingDate': '2022-06-02T00:00:00'}

In [37]:
drd.to_csv("previous_data_release_dates.csv", index=False)

In [43]:
pd.read_csv("previous_data_release_dates.csv",
           usecols=["commodityCode", "marketYear", "releaseTimeStamp"],
           parse_dates=["releaseTimeStamp"])

,commodityCode,marketYear,releaseTimeStamp
0,101,2023,2023-03-16
1,102,2023,2023-03-16
2,103,2023,2023-03-16
3,104,2023,2023-03-16
4,105,2023,2023-03-16
...,...,...,...
1028,1602,1998,1999-01-07
1029,1603,1998,1999-01-07
1030,1606,1998,1999-01-07
1031,1607,1998,1999-01-07
